In [1]:
import MyDL


In [5]:

a = MyDL.MyTensor([[1,2,3], [4,5,6]])
b = MyDL.MyTensor([1,2,3])
c = a + b
d = c.sum()
d.backward()
a.grad

array([[1, 1, 1],
       [1, 1, 1]])

In [9]:
a = MyDL.MyTensor([[1,2,3], [4,5,6]])
b = a.up_dim((0,2))
c = b.sum()
c.backward()
a.grad

array([[1, 1, 1],
       [1, 1, 1]])

In [1]:
import numpy as np
from numpy.lib.stride_tricks import as_strided
import MyDL
import MyDL.nn

np.random.seed(42)

conv_layer = MyDL.nn.layers.conv2D(in_channels=3, out_channels=5, kernel_size=4, stride=2, padding=(2, 1, 2, 1), bias=True)

X = np.random.rand(8, 3, 7, 7)   # 输入通道 C_in = 3
X = MyDL.MyTensor(X)
x_out = conv_layer(X)
x_out.shape
y = x_out.sum()
y.backward()


print(conv_layer.kernel.grad)


[[[[35.93753183 37.29460471 49.42811795 37.29460471]
   [39.20064628 33.93794498 51.94706724 33.93794498]
   [46.7024079  48.83980042 63.52849474 48.83980042]
   [39.20064628 33.93794498 51.94706724 33.93794498]]

  [[37.00600487 30.52571792 49.78620364 30.52571792]
   [37.38967876 36.21029921 48.37215612 36.21029921]
   [49.19773513 42.39234056 65.33942044 42.39234056]
   [37.38967876 36.21029921 48.37215612 36.21029921]]

  [[35.78772308 33.51175896 47.93191088 33.51175896]
   [35.71315127 38.77984207 47.3070206  38.77984207]
   [48.880439   47.95750157 63.62501306 47.95750157]
   [35.71315127 38.77984207 47.3070206  38.77984207]]]


 [[[35.93753183 37.29460471 49.42811795 37.29460471]
   [39.20064628 33.93794498 51.94706724 33.93794498]
   [46.7024079  48.83980042 63.52849474 48.83980042]
   [39.20064628 33.93794498 51.94706724 33.93794498]]

  [[37.00600487 30.52571792 49.78620364 30.52571792]
   [37.38967876 36.21029921 48.37215612 36.21029921]
   [49.19773513 42.39234056 65.33942

In [1]:
import numpy as np
X = np.random.rand(8, 3, 7, 7)

In [10]:
# 单通道卷积
import numpy as np
from numpy.lib.stride_tricks import as_strided

def conv2d_strided(X, K, padding=0, stride=1):
    """
    用 as_strided 实现高效的二维卷积（单通道）
    参数：
        X : 原始图像 (H, W)
        K : 卷积核 (kH, kW)
        padding : 填充边界（默认0）
        stride : 步幅（默认1）
    返回：
        Y : 卷积结果
    """
    X = np.array(X)
    K = np.array(K)

    H, W = X.shape
    kH, kW = K.shape

    # 添加 zero-padding
    if padding > 0:
        X_padded = np.pad(X, ((padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape

    # 输出尺寸
    out_h = (H_p - kH) // stride + 1
    out_w = (W_p - kW) // stride + 1

    # 构造滑动窗口视图：(out_h, out_w, kH, kW)
    shape = (out_h, out_w, kH, kW)
    strides = (
        X_padded.strides[0] * stride,
        X_padded.strides[1] * stride,
        X_padded.strides[0],
        X_padded.strides[1],
    )
    X_strided = as_strided(X_padded, shape=shape, strides=strides)

    # 逐窗口点乘卷积核并求和（向量化）
    Y = np.einsum('ijkl,kl->ij', X_strided, K)

    return Y

X = np.array([
    [1, 2, 3, 0],
    [0, 1, 2, 1],
    [1, 0, 2, 1],
    [2, 1, 3, 0]
])

K = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

Y = conv2d_strided(X, K, padding=1, stride=1)
print(Y)


[[-3 -4  2  5]
 [-3 -5  1  7]
 [-2 -4  0  7]
 [-1 -2  0  5]]


In [13]:
# 多通道卷积
import numpy as np
from numpy.lib.stride_tricks import as_strided

def conv2d_multi_channel(X, K, padding=0, stride=1):
    """
    多通道二维卷积实现（无for循环）
    参数：
        X: 输入图像 (C_in, H, W)
        K: 卷积核 (C_out, C_in, kH, kW)
        padding: 零填充大小
        stride: 步幅
    返回：
        Y: 输出特征图 (C_out, H_out, W_out)
    """
    C_in, H, W = X.shape
    C_out, C_in_k, kH, kW = K.shape
    assert C_in == C_in_k, "卷积核的输入通道数必须等于输入图像的通道数"

    # padding 每个通道
    if padding > 0:
        X_padded = np.pad(X, ((0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape[1], X_padded.shape[2]

    H_out = (H_p - kH) // stride + 1
    W_out = (W_p - kW) // stride + 1

    # 滑动窗口展开：输出形状 (C_in, H_out, W_out, kH, kW)
    strides = X_padded.strides
    new_shape = (C_in, H_out, W_out, kH, kW)
    new_strides = (
        strides[0],
        strides[1] * stride,
        strides[2] * stride,
        strides[1],
        strides[2]
    )
    X_strided = as_strided(X_padded, shape=new_shape, strides=new_strides)

    # 卷积操作：K 是 (C_out, C_in, kH, kW)，X 是 (C_in, H_out, W_out, kH, kW)
    # 使用einsum进行高效批量计算
    Y = np.einsum('oimn,ihwmn->ohw', K, X_strided)

    return Y

X = np.random.rand(3, 5, 5)   # 输入通道 C_in = 3
K = np.random.rand(4, 3, 3, 3) # 输出通道 C_out = 4，卷积核 3x3

Y = conv2d_multi_channel(X, K, padding=1, stride=1)

print(Y.shape)  # 应输出 (4, 5, 5)


(4, 5, 5)


In [8]:
# 多通道卷积
import numpy as np
from typing import Union, Tuple
from numpy.lib.stride_tricks import as_strided

def conv2d_multi_channel(X, K, padding:Union[int, Tuple[int, int, int, int]]=0, 
                         stride:Union[int, Tuple[int, int]]=1):
    """
    多通道二维卷积实现（无for循环）
    参数：
        X: 输入图像 (batch, C_in, H, W)
        K: 卷积核 (C_out, C_in, kH, kW)
        padding: 零填充大小，整数或元组(up, down, left, right)
        stride: 步幅
    返回：
        Y: 输出特征图 (C_out, H_out, W_out)
    """

    batch, C_in, H, W = X.shape
    C_out, C_in_k, kH, kW = K.shape
    assert C_in == C_in_k, "卷积核的输入通道数必须等于输入图像的通道数"

    # padding 每个通道
    if isinstance(padding, int):
        X_padded = np.pad(X, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = np.pad(X, ((0, 0), (0, 0), (padding[0], padding[1]), (padding[2], padding[3])), mode='constant')
    
    if isinstance(stride, int):
        stride = (stride, stride)

    H_p, W_p = X_padded.shape[2], X_padded.shape[3]

    H_out = (H_p - kH) // stride[0] + 1
    W_out = (W_p - kW) // stride[1] + 1

    # 滑动窗口展开：输出形状 (C_in, H_out, W_out, kH, kW)
    strides = X_padded.strides
    new_shape = (batch, C_in, H_out, W_out, kH, kW)
    new_strides = (
        strides[0],
        strides[1],
        strides[2] * stride[0],
        strides[3] * stride[1],
        strides[2],
        strides[3]
    )
    X_strided = as_strided(X_padded, shape=new_shape, strides=new_strides)

    # 卷积操作：K 是 (C_out, C_in, kH, kW)，X 是 (batch, C_in, H_out, W_out, kH, kW)
    # 使用einsum进行高效批量计算
    Y = np.einsum('oimn,bihwmn->bohw', K, X_strided)

    return Y


X = np.random.rand(8, 3, 7, 7)   # 输入通道 C_in = 3
K = np.random.rand(5, 3, 4, 4)   # c_in, c_out, kh, kw

Y = conv2d_multi_channel(X, K, padding=(2,1,2,1), stride=(2,2))

print(X.shape)
print(Y.shape)  # 应输出 (8, 4, 5, 5)

(8, 3, 7, 7)
(8, 5, 4, 4)


In [50]:
import numpy as np
a = np.array([[1,2,3],[4,5,6]])
a[:, ::2] = 0
a

array([[0, 2, 0],
       [0, 5, 0]])